# Structs
Rust has three kinds of struct types, named-field, tuple-like, and unit-like, which differ in how you refer to their components: a named-field struct gives a name to each com‐ ponent, whereas a tuple-like struct identifies them by the order in which they appear. Unit-like structs have no components at all; these are not common, but more useful than you might think.

## Named-Field Structs
The definition of a named-field struct type looks like this:

In [6]:
/// A rectangle of eight-bit grayscale pixels.
struct GrayscaleMap { 
    pixels: Vec<u8>, 
    size: (usize, usize)
}
fn new_map(size: (usize, usize), pixels: Vec<u8>) -> GrayscaleMap { 
    assert_eq!(pixels.len(), size.0 * size.1);
    GrayscaleMap { pixels, size }
}

let width = 1024;
let height = 576;
let image = GrayscaleMap {
    pixels: vec![0; width * height],
    size: (width, height)
};
assert_eq!(image.size, (1024, 576));
assert_eq!(image.pixels.len(), 1024 * 576);

Another examplem:

In [8]:
// In this game, brooms are monsters. You'll see.
struct Broom { name: String,
    height: u32,
    health: u32,
    position: (f32, f32, f32), 
    intent: BroomIntent
}
/// Two possible alternatives for what a `Broom` could be working on.
#[derive(Copy, Clone)]
enum BroomIntent { FetchWater, DumpWater }

// Receive the input Broom by value, taking ownership.
fn chop(b: Broom) -> (Broom, Broom) {
    // Initialize `broom1` mostly from `b`, changing only `height`. Since 
    // `String` is not `Copy`, `broom1` takes ownership of `b`'s name. 
    let mut broom1 = Broom { height: b.height / 2, .. b };
    // Initialize `broom2` mostly from `broom1`. Since `String` is not 
    // `Copy`, we must clone `name` explicitly.
    let mut broom2 = Broom { name: broom1.name.clone(), .. broom1 };
    // Give each fragment a distinct name.
    broom1.name.push_str(" I");
    broom2.name.push_str(" II");
    (broom1, broom2)
}

let hokey = Broom {
    name: "Hokey".to_string(), 
    height: 60,
    health: 100,
    position: (100.0, 200.0, 0.0), 
    intent: BroomIntent::FetchWater
};

let (hokey1, hokey2) = chop(hokey); 
assert_eq!(hokey1.name, "Hokey I"); 
assert_eq!(hokey1.height, 30); 
assert_eq!(hokey1.health, 100);
assert_eq!(hokey2.name, "Hokey II");
assert_eq!(hokey1.height, 30);
assert_eq!(hokey2.health, 100);

## Tuple-Like Structs


In [9]:
struct Bounds(usize, usize);
let image_bounds = Bounds(1024, 768);
assert_eq!(image_bounds.0 * image_bounds.1, 786432);

Individual elements of a tuple-like struct may be public or not:
```Rust
pub struct Bounds(pub usize, pub usize);
```
The expression Bounds(1024, 768) looks like a function call, and in fact it is: defining the type also implicitly defines a function:
```Rust
fn Bounds(elem0: usize, elem1: usize) -> Bounds { ... }
```

Tuple-like structs are good for newtypes, structs with a single component that you define to get stricter type checking. For example, if you are working with ASCII-only text, you might define a newtype like this:
```Rust
struct Ascii(Vec<u8>);
```

## Unit-Like Structs
The third kind of struct is a little obscure: it declares a struct type with no elements at all:
```Rust
struct Onesuch;
```
A value of such a type occupies no memory, much like the unit type `()`. Rust doesn’t bother actually storing unit-like struct values in memory or generating code to oper‐ ate on them, because it can tell everything it might need to know about the value from its type alone. But logically, an empty struct is a type with values like any other—or more precisely, a type of which there is only a single value:
```Rust
let o = Onesuch;
```

## Struct Layout


In [10]:
struct GrayscaleMap { 
    pixels: Vec<u8>,
    size: (usize, usize) 
}

![image.png](asset/ch9/1.png)

Unlike C and C++, Rust doesn’t make specific promises about how it will order a struct’s fields or elements in memory; this diagram shows only one possible arrangement. However, Rust does promise to store fields’ values directly in the struct’s block of memory.

You can ask Rust to lay out structures in a way compatible with C and C++, using the `#[repr(C)]` attribute.

## Defining Methods with impl
An impl block is simply a collection of fn definitions, each of which becomes a method on the struct type named at the top of the block.

In [11]:
/// A first-in, first-out queue of characters.
pub struct Queue {
    older: Vec<char>, // older elements, eldest last. 
    younger: Vec<char> // younger elements, youngest last.
}
impl Queue {
    /// Push a character onto the back of a queue. 
    pub fn push(&mut self, c: char) {
        self.younger.push(c);
    }
    /// Pop a character off the front of a queue. Return `Some(c)` if there /// was a character to pop, or `None` if the queue was empty.
    pub fn pop(&mut self) -> Option<char> {
        if self.older.is_empty() {
            if self.younger.is_empty() {
                return None; 
            }
            // Bring the elements in younger over to older, and put them in // the promised order.
            use std::mem::swap;
            swap(&mut self.older, &mut self.younger); 
            self.older.reverse();
        }
        // Now older is guaranteed to have something. Vec's pop method // already returns an Option, so we're set.
        self.older.pop()
    } 
}

Functions defined in an impl block are called associated functions, since they’re associated with a specific type. The opposite of an associated function is a free function, one that is not defined as an impl block’s item.

Rust passes a method the value it’s being called on as its first argument, which must have the special name self. Since self’s type is obviously the one named at the top of the impl block, or a reference to that, Rust lets you omit the type, and write `self`, `&self`, or `&mut self` as shorthand for `self: Queue`, `self: &Queue`, or `self: &mut Queue`. You can use the longhand forms if you like, but almost all Rust code uses the shorthand, as shown before.

### Passing Self as a Box, Rc, or Arc
Putting `Rc<Self>` into the signature of the append_to method makes the caller aware of Node’s requirements. The caller is then able to minimize allocation and reference- counting activity given its own needs:
* If it can pass ownership of the Rc, it simply hands over the pointer.
* If it needs to retain ownership of an Rc, it just bumps the reference count.
* Only if it owns the Node itself must it call `Rc::new` to allocate heap space and move the Node into it. Since parent will insist on referring to its children via `Rc<Node>` pointers, this was going to be necessary eventually.
Again, for most methods, `&self`, `&mut self`, and `self` (by value) are all you need. But if a method’s purpose is to affect the ownership of the value, using other pointer types for self can be just the right thing.


In [42]:
use std::rc::Rc;
struct Node { 
    tag: String,
    children: Vec<Rc<Node>>
}
impl Node {
    fn new(tag: &str) -> Node {
        Node {
            tag: tag.to_string(), 
            children: vec![],
        } 
    }
}

impl Node {
    fn append_to(self: Rc<Self>, parent: &mut Node) {
        parent.children.push(self);
    }
}

let mut parent: Node = Node::new("parent"); 
{
    let shared_node: Rc<Node> = Rc::new(Node::new("first")); 
    shared_node.append_to(&mut parent);
}
println!("{}", &(parent.children[0].tag));

first


实际上是可以用引用来实现的，但是通过引用实现的话,生命周期会被限制地相当狭小，具体可以参照本人拓展原书的上方代码块和实现的下方代码块。

In [33]:
struct Node<'a> {
    tag: String,
    children: Vec<&'a Node<'a>>,
}

impl<'a> Node<'a> {
    fn new(tag: &str) -> Node<'a> {
        Node {
            tag: tag.to_string(),
            children: vec![],
        }
    }

    fn append_to(&'a self, parent: &'a mut Node<'a>) {
        parent.children.push(self);
    }
}

{
    let mut parent: Node = Node::new("parent"); 
    let mut shared_node: Node = Node::new("first"); 
    shared_node.append_to(&mut parent);
    println!("{}", shared_node.tag);
};

first


### Type-Associated Functions

An impl block for a given type can also define functions that don’t take self as an argument at all. These are still associated functions, since they’re in an impl block, but they’re not methods, since they don’t take a self argument. To distinguish them from methods, we call them type-associated functions.
```Rust
impl Queue {
    pub fn new() -> Queue {
        Queue { older: Vec::new(), younger: Vec::new() }
    }
}
```

It’s conventional in Rust for constructor functions to be named `new`; we’ve already seen `Vec::new`, `Box::new`, `HashMap::new`, and others. But there’s nothing special about the name new. It’s not a keyword, and types often have other associated functions that serve as constructors, like `Vec::with_capacity`.

## Associated Consts
Another feature of languages like C# and Java that Rust adopts in its type system is the idea of values associated with a type, rather than a specific instance of that type. In Rust, these are known as `associated consts`.

As the name implies, associated consts are constant values. They’re often used to specify commonly used values of a type. For instance, you could define a two-dimensional vector for use in linear algebra with an associated unit vector:


In [53]:
pub struct Vector2 { 
    x: f32,
    y: f32, 
}

impl Vector2 {
    const ZERO: Vector2 = Vector2 { x: 0.0, y: 0.0 }; 
    const UNIT: Vector2 = Vector2 { x: 1.0, y: 0.0 };
    fn scaled_by (mut self, scale: f32) -> Vector2{
        self.x *= scale;
        self.y *= scale;
        self
    }
}

In [56]:
let scaled = Vector2::UNIT.scaled_by(2.0);
println!("x: {}, y: {}", scaled.x, scaled.y);

x: 2, y: 0


Nor does an associated const have to be of the same type as the type it’s associated with; we could use this feature to add IDs or names to types. For example, if there were several types similar to `Vector2` that needed to be written to a file and then loaded into memory later, an associated const could be used to add names or numeric IDs that could be written next to the data to identify its type:

In [57]:
impl Vector2 {
    const NAME: &'static str = "Vector2"; 
    const ID: u32 = 18;
}

## Generic Structs
Fortunately, Rust structs can be generic, meaning that their definition is a template into which you can plug whatever types you like. For example, here’s a definition for Queue that can hold values of any type:

In [4]:
pub struct Queue<T> { 
    older: Vec<T>,
    younger: Vec<T>
}

impl<T> Queue<T> {
    pub fn new() -> Queue<T> {
        // equivalent due to the auto reference of rust
        // Queue { older: Vec::new(), younger: Vec::new() }
        Queue { older: Vec::<T>::new(), younger: Vec::<T>::new() }
    }
    pub fn push(&mut self, t: T) { 
        self.younger.push(t);
    }
    pub fn is_empty(&self) -> bool { 
        self.older.is_empty() && self.younger.is_empty()
    }
}
let mut q = Queue::<char>::new();

In [5]:
let mut s = Queue::new(); 
let mut r = Queue::new();

s.push("CAD"); // apparently a Queue<&'static str> 
r.push(0.74); // apparently a Queue<f64>

s.push("BTC"); // Bitcoins per USD, 2019-6
r.push(13764.0); // Rust fails to detect irrational exuberance

## Structs with Lifetime Parameters
If a struct type contains references, you must name those references’ lifetimes. For example, here’s a structure that might hold references to the greatest and least elements of some slice:

In [9]:
struct Extrema<'elt> { 
    greatest: &'elt i32, 
    least: &'elt i32
}

fn find_extrema<'s>(slice: &'s [i32]) -> Extrema<'s> { 
    let mut greatest = &slice[0];
    let mut least = &slice[0];
    for i in 1..slice.len() {
        if slice[i] < *least { least = &slice[i]; } 
        if slice[i] > *greatest { greatest = &slice[i]; }
    }
    Extrema { greatest, least }
}

{
    let a = [0,-3,0,15,48]; 
    let e = find_extrema(&a); 
    assert_eq!(*e.least, -3); 
    assert_eq!(*e.greatest, 48);
};

Because it’s so common for the return type to use the same lifetime as an argument, Rust lets us omit the lifetimes when there’s one obvious candidate. We could also have written find_extrema’s signature like this, with no change in meaning:
```Rust
fn find_extrema(slice: &[i32]) -> Extrema { 
    ...
}
```
Granted, we might have meant `Extrema<'static>`, but that’s pretty unusual. Rust provides a shorthand for the common case.

## Deriving Common Traits for Struct Types
Structs can be very easy to write:
```Rust
struct Point { 
    x: f64,
    y: f64 
}
```
However, if you were to start using this Point type, you would quickly notice that it’s a bit of a pain. As written, Point is not copyable or cloneable. You can’t print it with println!`("{:?}", point);` and it does not support the `==` and `!=` operators.

Each of these features has a name in Rust—`Copy`, `Clone`, `Debug`, and `PartialEq`. They are called traits. In the case of these standard traits, and several others, you don’t need to implement them by hand unless you want some kind of custom behavior. Rust can automatically implement them for you, with mechanical accuracy. Just add a `#[derive]` attribute to the struct:
```Rust
#[derive(Copy, Clone, Debug, PartialEq)]
struct Point { 
    x: f64,
    y: f64 
}
```
Each of these traits can be implemented automatically for a struct, provided that each of its fields implements the trait. We can ask Rust to derive `PartialEq` for `Point` because its two fields are both of type `f64`, which already implements `PartialEq`.

Rust can also derive PartialOrd, which would add support for the comparison operators `<`, `>`, `<=`, and `>=`. We haven’t done so here, because comparing two points to see if one is “less than” the other is actually a pretty weird thing to do. There’s no one conventional order on points. So we choose not to support those operators for Point values. Cases like this are one reason that Rust makes us write the `#[derive]` attribute rather than automatically deriving every trait it can. Another reason is that implementing a trait is automatically a public feature, so copyability, cloneability, and so forth are all part of your struct’s public API and should be chosen deliberately.

## Interior Mutability
What we need is a little bit of mutable data inside an otherwise immutable value. This is called _interior mutability_. Rust offers several flavors of it; in this section, we’ll discuss the two most straightforward types: `Cell<T>` and `RefCell<T>`, both in the `std::cell` module.

In [20]:
use std::cell::{RefCell, Ref, RefMut};
let ref_cell: RefCell<String> = RefCell::new("hello".to_string());
{
    let r: Ref<String> = ref_cell.borrow(); // ok, returns a Ref<String> 
    let count: usize = r.len(); // ok, returns "hello".len() assert_eq!(count, 5);
    // let mut w: RefMut<String> = ref_cell.borrow_mut(); // panic: already borrowed w.push_str(" world");
};
{
    let mut w: RefMut<String> = ref_cell.borrow_mut(); // panic: already borrowed w.push_str(" world");
};

Cells are easy to use. Having to call `.get()` and `.set()` or `.borrow()` and `.borrow_mut()` is slightly awkward, but that’s just the price we pay for bending the rules. The other drawback is less obvious and more serious: cells—and any types that contain them—are not thread-safe. Rust therefore will not allow multiple threads to access them at once.